In [99]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, VotingRegressor, BaggingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [100]:
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/sample_submission.csv')
bookings_data = pd.read_csv('../data/bookings_data.csv')
bookings = pd.read_csv('../data/bookings.csv')
hotel_data = pd.read_csv('../data/hotels_data.csv')
customer_data = pd.read_csv('../data/customer_data.csv')
payments_data = pd.read_csv('../data/payments_data.csv')

In [101]:
# convert payment type to numeric using sk preprocessing label encoder
le = LabelEncoder()
le.fit(payments_data['payment_type'])
payments_data['payment_type'] = le.transform(payments_data['payment_type'])

# add 1 to the payment type to avoid 0 values
payments_data['payment_type'] = payments_data['payment_type'] + 1

payments_data.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886.000000,103886.000000,103886.000000,103886.000000
unique,99440,NaN,NaN,NaN,NaN
top,d1b0e818e3ccc5cb0e39231352fa65da,NaN,NaN,NaN,NaN
freq,29,NaN,NaN,NaN,NaN
mean,NaN,1.092679,1.618043,2.853349,154.100380
std,NaN,0.706584,1.133229,2.687051,217.494064
min,NaN,1.000000,1.000000,0.000000,0.000000
25%,NaN,1.000000,1.000000,1.000000,56.790000
50%,NaN,1.000000,1.000000,1.000000,100.000000
75%,NaN,1.000000,2.000000,4.000000,171.837500


In [102]:
# keep only entries with payment_sequential as 1
payments_data_unique = payments_data[payments_data['payment_sequential'] == 1]
payments_data_repeat = payments_data[payments_data['payment_sequential'] > 1]

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.0,99360.000000,99360.000000,99360.000000
unique,99360,NaN,NaN,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN
mean,NaN,1.0,1.478422,2.929640,158.336774
std,NaN,0.0,0.918534,2.714947,220.511857
min,NaN,1.0,1.000000,1.000000,0.000000
25%,NaN,1.0,1.000000,1.000000,59.947500
50%,NaN,1.0,1.000000,2.000000,103.330000
75%,NaN,1.0,1.000000,4.000000,175.110000


In [103]:
# sort payments_data_repeat by payment_sequential
payments_data_repeat = payments_data_repeat.sort_values(by=['payment_sequential'], ascending=True)

payments_data_repeat.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,4526,4526.000000,4526.000000,4526.000000,4526.000000
unique,3039,NaN,NaN,NaN,NaN
top,d1b0e818e3ccc5cb0e39231352fa65da,NaN,NaN,NaN,NaN
freq,28,NaN,NaN,NaN,NaN
mean,NaN,3.127265,4.683164,1.178524,61.098164
std,NaN,2.670762,1.063888,0.989643,96.149183
min,NaN,2.000000,1.000000,0.000000,0.000000
25%,NaN,2.000000,5.000000,1.000000,14.905000
50%,NaN,2.000000,5.000000,1.000000,31.920000
75%,NaN,3.000000,5.000000,1.000000,71.230000


In [104]:
# making payment data unique for each booking_id by adding the payments made by other methods to primary payment method
columns = ['payment_value', 'payment_installments', 'payment_type']

for payment_data_repeat in payments_data_repeat.itertuples():
    booking_id = payment_data_repeat.booking_id
    payment_data_unique = payments_data_unique[payments_data_unique['booking_id'] == booking_id]
    for column in columns:
        new_value = payment_data_unique[column] + payment_data_repeat.__getattribute__(column)
        payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, column] = new_value
    payments_data_unique.loc[payments_data_unique['booking_id'] == booking_id, 'payment_sequential'] = payment_data_repeat.payment_sequential

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.000000,99360.000000,99360.000000,99360.000000
unique,99360,NaN,NaN,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN
mean,NaN,1.044726,1.690378,2.981310,160.984584
std,NaN,0.381293,2.149129,2.741804,221.998050
min,NaN,1.000000,1.000000,1.000000,0.000000
25%,NaN,1.000000,1.000000,1.000000,62.010000
50%,NaN,1.000000,1.000000,2.000000,105.290000
75%,NaN,1.000000,2.000000,4.000000,176.932500


In [105]:
# make columns values average based on payment_sequential
for column in columns:
    payments_data_unique[column] = payments_data_unique[column] / payments_data_unique['payment_sequential']

payments_data_unique.describe(include='all')

,booking_id,payment_sequential,payment_type,payment_installments,payment_value
count,99360,99360.000000,99360.000000,99360.000000,99360.000000
unique,99360,NaN,NaN,NaN,NaN
top,6f3fe1789b1e8b2acac839d17b81ef22,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN
mean,NaN,1.044726,1.527026,2.915060,158.311082
std,NaN,0.381293,0.949964,2.700228,219.239331
min,NaN,1.000000,1.000000,1.000000,0.000000
25%,NaN,1.000000,1.000000,1.000000,60.180000
50%,NaN,1.000000,1.000000,2.000000,103.310000
75%,NaN,1.000000,2.000000,4.000000,174.970000


In [106]:
bookings_data.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees
count,112650,112650.000000,112650,112650,112650,112650.000000,112650.000000
unique,98666,NaN,32951,3095,93318,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,1a7b8ebd4e68314663afaca2eb7d00ea,19b47992c3666cc44a7e94c06560211a,2017-07-21 18:25:23,NaN,NaN
freq,21,NaN,527,2033,21,NaN,NaN
mean,NaN,1.197834,NaN,NaN,NaN,120.653739,19.990320
std,NaN,0.705124,NaN,NaN,NaN,183.633928,15.806405
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000
25%,NaN,1.000000,NaN,NaN,NaN,39.900000,13.080000
50%,NaN,1.000000,NaN,NaN,NaN,74.990000,16.260000
75%,NaN,1.000000,NaN,NaN,NaN,134.900000,21.150000


In [107]:
bookings_data_new = bookings_data.merge(hotel_data, on='hotel_id', how='left')

bookings_data_new.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,112650,112650.000000,112650,112650,112650,112650.000000,112650.000000,111047.000000,111047.000000,111047.000000,111047.000000
unique,98666,NaN,32951,3095,93318,NaN,NaN,NaN,NaN,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,1a7b8ebd4e68314663afaca2eb7d00ea,19b47992c3666cc44a7e94c06560211a,2017-07-21 18:25:23,NaN,NaN,NaN,NaN,NaN,NaN
freq,21,NaN,527,2033,21,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.197834,NaN,NaN,NaN,120.653739,19.990320,28.438706,48.775978,787.867029,2.209713
std,NaN,0.705124,NaN,NaN,NaN,183.633928,15.806405,22.608198,10.025581,652.135608,1.721438
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000
25%,NaN,1.000000,NaN,NaN,NaN,39.900000,13.080000,10.000000,42.000000,348.000000,1.000000
50%,NaN,1.000000,NaN,NaN,NaN,74.990000,16.260000,27.000000,52.000000,603.000000,1.000000
75%,NaN,1.000000,NaN,NaN,NaN,134.900000,21.150000,38.000000,57.000000,987.000000,3.000000


In [108]:
# split bookings_data into unique and repeat bookings
bookings_data_unique = bookings_data_new[bookings_data_new['booking_sequence_id'] == 1]
bookings_data_repeat = bookings_data_new[bookings_data_new['booking_sequence_id'] > 1]

bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,98666,98666.0,98666,98666,98666,98666.000000,98666.000000,97250.000000,97250.000000,97250.000000,97250.000000
unique,98666,NaN,31881,3088,93001,NaN,NaN,NaN,NaN,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2017-12-07 04:39:05,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,439,1844,6,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.0,NaN,NaN,NaN,125.964327,20.201927,28.856422,48.846386,794.161398,2.250591
std,NaN,0.0,NaN,NaN,NaN,191.375106,15.909873,22.666001,9.999239,654.751953,1.747095
min,NaN,1.0,NaN,NaN,NaN,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000
25%,NaN,1.0,NaN,NaN,NaN,41.505000,13.310000,10.000000,42.000000,349.000000,1.000000
50%,NaN,1.0,NaN,NaN,NaN,79.000000,16.360000,28.000000,52.000000,607.000000,2.000000
75%,NaN,1.0,NaN,NaN,NaN,139.900000,21.230000,38.000000,57.000000,996.000000,3.000000


In [109]:
# sort bookings_data_repeat by booking_sequence_id
bookings_data_repeat = bookings_data_repeat.sort_values(by=['booking_sequence_id'], ascending=True)

bookings_data_repeat.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,13984,13984.000000,13984,13984,13984,13984.000000,13984.000000,13797.000000,13797.000000,13797.000000,13797.000000
unique,9803,NaN,6593,1548,9828,NaN,NaN,NaN,NaN,NaN,NaN
top,f5f79c56e9e4120aec44ef8272b63d03,NaN,46682990de24d770e7f83d422879e10f,0176fa81dab994f90235231001f50f92,2017-07-21 18:25:23,NaN,NaN,NaN,NaN,NaN,NaN
freq,20,NaN,144,542,20,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.593678,NaN,NaN,NaN,83.184167,18.497299,25.494383,48.279698,743.500326,1.921577
std,NaN,1.334476,NaN,NaN,NaN,107.874536,14.971970,21.973179,10.195912,631.633730,1.497150
min,NaN,2.000000,NaN,NaN,NaN,0.850000,0.000000,1.000000,6.000000,8.000000,1.000000
25%,NaN,2.000000,NaN,NaN,NaN,30.000000,11.850000,7.000000,42.000000,338.000000,1.000000
50%,NaN,2.000000,NaN,NaN,NaN,56.000000,15.560000,17.000000,51.000000,557.000000,1.000000
75%,NaN,3.000000,NaN,NaN,NaN,99.900000,20.160000,35.000000,57.000000,919.000000,2.000000


In [110]:
# merging bookings_data for each booking_id
columns = ['price', 'agent_fees', 'hotel_category', 'hotel_name_length', 'hotel_description_length', 'hotel_photos_qty', 'booking_expiry_date']

for booking_data_repeat in bookings_data_repeat.itertuples():
    bookings_id = booking_data_repeat.booking_id
    booking_data_unique = bookings_data_unique[bookings_data_unique['booking_id'] == bookings_id]
    for column in columns:
        new_value = booking_data_unique[column] + booking_data_repeat.__getattribute__(column)
        bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, column] = new_value
    bookings_data_unique.loc[bookings_data_unique['booking_id'] == bookings_id, 'booking_sequence_id'] = booking_data_repeat.booking_sequence_id

In [ ]:
bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,98666,98666.000000,98666,98666,98666,98666.000000,98666.000000,97215.000000,97215.000000,97215.000000,97215.000000
unique,98666,NaN,31881,3088,93001,NaN,NaN,NaN,NaN,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2017-12-07 04:39:05,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,439,1844,6,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.141731,NaN,NaN,NaN,137.754076,22.823562,32.453737,55.673188,899.198961,2.522162
std,NaN,0.538452,NaN,NaN,NaN,210.645145,21.650909,30.513973,28.456340,905.810601,2.228737
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000
25%,NaN,1.000000,NaN,NaN,NaN,45.900000,13.850000,11.000000,44.000000,375.000000,1.000000
50%,NaN,1.000000,NaN,NaN,NaN,86.900000,17.170000,28.000000,53.000000,653.000000,2.000000
75%,NaN,1.000000,NaN,NaN,NaN,149.900000,24.040000,45.000000,58.000000,1103.500000,3.000000


In [ ]:
# make entries in bookings_data_unique by taking average of the values based in booking_sequence_id
columns = ['hotel_category', 'hotel_name_length', 'hotel_description_length', 'hotel_photos_qty', 'booking_expiry_date']

for column in columns:
    bookings_data_unique[column] = bookings_data_unique[column] / bookings_data_unique['booking_sequence_id']

bookings_data_unique.describe(include='all')

,booking_id,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,98666,98666.000000,98666,98666,98666,98666.000000,98666.000000,97215.000000,97215.000000,97215.000000,97215.000000
unique,98666,NaN,31881,3088,93001,NaN,NaN,NaN,NaN,NaN,NaN
top,242fe8c5a6d1ba2dd792cb1621400010,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2017-12-07 04:39:05,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,439,1844,6,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.141731,NaN,NaN,NaN,137.754076,22.823562,28.857976,48.843101,794.016654,2.249931
std,NaN,0.538452,NaN,NaN,NaN,210.645145,21.650909,22.628233,9.958101,652.914402,1.739707
min,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000
25%,NaN,1.000000,NaN,NaN,NaN,45.900000,13.850000,10.000000,42.000000,351.000000,1.000000
50%,NaN,1.000000,NaN,NaN,NaN,86.900000,17.170000,28.000000,52.000000,608.000000,2.000000
75%,NaN,1.000000,NaN,NaN,NaN,149.900000,24.040000,38.000000,57.000000,996.000000,3.000000


In [ ]:
bookings.describe(include='all')

,booking_id,customer_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date
count,99441,99441,99441,99441,99281,96476
unique,99441,99441,8,98875,90733,95664
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46
freq,1,1,96478,3,9,3


In [ ]:
# merge bookings and bookings_data as bookings_df
bookings_df = pd.merge(bookings, bookings_data_unique, on='booking_id', how='left')

bookings_df.describe(include='all')

,booking_id,customer_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty
count,99441,99441,99441,99441,99281,96476,98666.000000,98666,98666,98666,98666.000000,98666.000000,97215.000000,97215.000000,97215.000000,97215.000000
unique,99441,99441,8,98875,90733,95664,NaN,31881,3088,93001,NaN,NaN,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,51297304e76186b10a928d9ef432eb62,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2018-06-11 03:31:04,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,96478,3,9,3,NaN,439,1844,6,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.141731,NaN,NaN,NaN,137.754076,22.823562,28.857976,48.843101,794.016654,2.249931
std,NaN,NaN,NaN,NaN,NaN,NaN,0.538452,NaN,NaN,NaN,210.645145,21.650909,22.628233,9.958101,652.914402,1.739707
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,45.900000,13.850000,10.000000,42.000000,351.000000,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,86.900000,17.170000,28.000000,52.000000,608.000000,2.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,149.900000,24.040000,38.000000,57.000000,996.000000,3.000000


In [ ]:
# merge bookings_df and customer_data as bookings_customer_df
bookings_customer_df = pd.merge(bookings_df, customer_data, on='customer_id', how='left')

# merge bookings_hotel_df and payments_data as bookings_payment_df
bookings_payment_df = pd.merge(bookings_customer_df, payments_data_unique, on='booking_id', how='left')

bookings_payment_df.drop(['customer_id'], axis=1, inplace=True)

bookings_payment_df.describe(include='all')

,booking_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,...,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,customer_unique_id,country,payment_sequential,payment_type,payment_installments,payment_value
count,99441,99441,99441,99281,96476,98666.000000,98666,98666,98666,98666.000000,...,97215.000000,97215.000000,97215.000000,97215.000000,99441,99441,99360.000000,99360.000000,99360.000000,99360.000000
unique,99441,8,98875,90733,95664,NaN,31881,3088,93001,NaN,...,NaN,NaN,NaN,NaN,96096,9,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,completed,2008-04-13 10:31:14,2008-03-01 04:14:10,2008-05-10 23:21:46,NaN,856965c36a24e339b605899a4788cb24,19b47992c3666cc44a7e94c06560211a,2018-06-11 03:31:04,NaN,...,NaN,NaN,NaN,NaN,f50201ccdcedfb9e2ac84558d50f5ead,Slovakia,NaN,NaN,NaN,NaN
freq,1,96478,3,9,3,NaN,439,1844,6,NaN,...,NaN,NaN,NaN,NaN,17,11212,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.141731,NaN,NaN,NaN,137.754076,...,28.857976,48.843101,794.016654,2.249931,NaN,NaN,1.044726,1.527026,2.915060,158.311082
std,NaN,NaN,NaN,NaN,NaN,0.538452,NaN,NaN,NaN,210.645145,...,22.628233,9.958101,652.914402,1.739707,NaN,NaN,0.381293,0.949964,2.700228,219.239331
min,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.850000,...,1.000000,5.000000,4.000000,1.000000,NaN,NaN,1.000000,1.000000,1.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,45.900000,...,10.000000,42.000000,351.000000,1.000000,NaN,NaN,1.000000,1.000000,1.000000,60.180000
50%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,86.900000,...,28.000000,52.000000,608.000000,2.000000,NaN,NaN,1.000000,1.000000,2.000000,103.310000
75%,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,149.900000,...,38.000000,57.000000,996.000000,3.000000,NaN,NaN,1.000000,2.000000,4.000000,174.970000


In [ ]:
cat_columns = ['seller_agent_id', 'booking_status', 'country', 'customer_unique_id', 'hotel_id']

for column in cat_columns:
    le = LabelEncoder()
    le.fit(bookings_payment_df[column])
    bookings_payment_df[column] = le.transform(bookings_payment_df[column])
    if column == 'booking_status' or column == 'country':
        bookings_payment_df[column] = bookings_payment_df[column] + 1

print(bookings_payment_df.dtypes)

booking_id                        object
booking_status                     int32
booking_create_timestamp          object
booking_approved_at               object
booking_checkin_customer_date     object
booking_sequence_id              float64
hotel_id                           int32
seller_agent_id                    int32
booking_expiry_date               object
price                            float64
agent_fees                       float64
hotel_category                   float64
hotel_name_length                float64
hotel_description_length         float64
hotel_photos_qty                 float64
customer_unique_id                 int32
country                            int32
payment_sequential               float64
payment_type                     float64
payment_installments             float64
payment_value                    float64
dtype: object


In [ ]:
date_columns = ['booking_create_timestamp', 'booking_approved_at', 'booking_checkin_customer_date', 'booking_expiry_date']

for date_column in date_columns:
    bookings_payment_df[date_column] = pd.to_datetime(bookings_payment_df[date_column])

# change approved-at to approved_at - create_timestamp
bookings_payment_df['booking_approved_at'] = bookings_payment_df['booking_approved_at'] - bookings_payment_df['booking_create_timestamp']
bookings_payment_df['booking_approved_at'] = bookings_payment_df['booking_approved_at'].dt.total_seconds()

# change expiry to expiry - checkin
bookings_payment_df['booking_expiry_date'] = bookings_payment_df['booking_expiry_date'] - bookings_payment_df['booking_checkin_customer_date']
bookings_payment_df['booking_expiry_date'] = bookings_payment_df['booking_expiry_date'].dt.total_seconds()

print(bookings_payment_df.dtypes)
bookings_payment_df.describe(include='all')

booking_id                               object
booking_status                            int32
booking_create_timestamp         datetime64[ns]
booking_approved_at                     float64
booking_checkin_customer_date    datetime64[ns]
booking_sequence_id                     float64
hotel_id                                  int32
seller_agent_id                           int32
booking_expiry_date                     float64
price                                   float64
agent_fees                              float64
hotel_category                          float64
hotel_name_length                       float64
hotel_description_length                float64
hotel_photos_qty                        float64
customer_unique_id                        int32
country                                   int32
payment_sequential                      float64
payment_type                            float64
payment_installments                    float64
payment_value                           

,booking_id,booking_status,booking_create_timestamp,booking_approved_at,booking_checkin_customer_date,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,...,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,customer_unique_id,country,payment_sequential,payment_type,payment_installments,payment_value
count,99441,99441.000000,99441,9.928100e+04,96476,98666.000000,99441.000000,99441.000000,9.647600e+04,98666.000000,...,97215.000000,97215.000000,97215.000000,97215.000000,99441.000000,99441.000000,99360.000000,99360.000000,99360.000000,99360.000000
unique,99441,NaN,98875,NaN,95664,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,NaN,2008-04-13 10:31:14,NaN,2008-05-10 23:21:46,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,3,NaN,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,2006-09-07 20:58:19,NaN,2006-10-14 13:29:32,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2008-10-19 17:13:18,NaN,2008-10-19 13:05:46,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.076166,NaN,3.750874e+04,NaN,1.141731,16030.302461,1417.288995,3.148433e+08,137.754076,...,28.857976,48.843101,794.016654,2.249931,48049.895224,5.011484,1.044726,1.527026,2.915060,158.311082
std,NaN,0.561226,NaN,9.373681e+04,NaN,0.538452,9287.168077,935.610297,8.691278e+05,210.645145,...,22.628233,9.958101,652.914402,1.739707,27758.278975,2.580036,0.381293,0.949964,2.700228,219.239331
min,NaN,1.000000,NaN,0.000000e+00,NaN,1.000000,0.000000,0.000000,2.975947e+08,0.850000,...,1.000000,5.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,NaN,3.000000,NaN,7.740000e+02,NaN,1.000000,7995.000000,526.000000,3.146086e+08,45.900000,...,10.000000,42.000000,351.000000,1.000000,23986.000000,3.000000,1.000000,1.000000,1.000000,60.180000


In [ ]:
bookings_payment_df.drop(['booking_create_timestamp', 'booking_checkin_customer_date'], axis=1, inplace=True)

In [ ]:
# take all columns
columns = bookings_payment_df.columns

# remove booking_id
columns = columns.drop(['booking_id'])

# change all null or nan values to mean of respective columns
for column in columns:
    mean = bookings_payment_df[column].mean()
    bookings_payment_df[column].fillna(mean, inplace=True)

In [ ]:
print(bookings_payment_df.dtypes)

booking_id                   object
booking_status                int32
booking_approved_at         float64
booking_sequence_id         float64
hotel_id                      int32
seller_agent_id               int32
booking_expiry_date         float64
price                       float64
agent_fees                  float64
hotel_category              float64
hotel_name_length           float64
hotel_description_length    float64
hotel_photos_qty            float64
customer_unique_id            int32
country                       int32
payment_sequential          float64
payment_type                float64
payment_installments        float64
payment_value               float64
dtype: object


In [ ]:
bookings_payment_df.describe(include='all')

,booking_id,booking_status,booking_approved_at,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,customer_unique_id,country,payment_sequential,payment_type,payment_installments,payment_value
count,99441,99441.000000,9.944100e+04,99441.000000,99441.000000,99441.000000,9.944100e+04,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.076166,3.750874e+04,1.141731,16030.302461,1417.288995,3.148433e+08,137.754076,22.823562,28.857976,48.843101,794.016654,2.249931,48049.895224,5.011484,1.044726,1.527026,2.915060,158.311082
std,NaN,0.561226,9.366137e+04,0.536350,9287.168077,935.610297,8.560723e+05,209.822693,21.566375,22.373529,9.846012,645.565179,1.720124,27758.278975,2.580036,0.381138,0.949577,2.699128,219.150021
min,NaN,1.000000,0.000000e+00,1.000000,0.000000,0.000000,2.975947e+08,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,NaN,3.000000,7.750000e+02,1.000000,7995.000000,526.000000,3.146300e+08,45.990000,13.900000,10.000000,43.000000,357.000000,1.000000,23986.000000,3.000000,1.000000,1.000000,1.000000,60.230000
50%,NaN,3.000000,1.239000e+03,1.000000,16249.000000,1342.000000,3.149849e+08,88.000000,17.270000,28.000000,51.000000,621.000000,2.000000,48053.000000,5.000000,1.000000,1.000000,2.000000,103.370000
75%,NaN,3.000000,5.233600e+04,1.000000,23920.000000,2260.000000,3.153140e+08,149.900000,23.920000,38.000000,57.000000,982.000000,3.000000,72088.000000,7.000000,1.000000,2.000000,4.000000,174.880000


In [ ]:
# scale date columns using StandardScaler
date_columns = ['booking_approved_at', 'booking_expiry_date']

scaled_columns = StandardScaler().fit_transform(bookings_payment_df[date_columns])

bookings_payment_df[date_columns] = scaled_columns

bookings_payment_df.describe(include='all')

,booking_id,booking_status,booking_approved_at,booking_sequence_id,hotel_id,seller_agent_id,booking_expiry_date,price,agent_fees,hotel_category,hotel_name_length,hotel_description_length,hotel_photos_qty,customer_unique_id,country,payment_sequential,payment_type,payment_installments,payment_value
count,99441,99441.000000,9.944100e+04,99441.000000,99441.000000,99441.000000,9.944100e+04,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000,99441.000000
unique,99441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,c54678b7cc49136f2d6af7e481f51cbd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.076166,-8.336860e-17,1.141731,16030.302461,1417.288995,-2.076880e-14,137.754076,22.823562,28.857976,48.843101,794.016654,2.249931,48049.895224,5.011484,1.044726,1.527026,2.915060,158.311082
std,NaN,0.561226,1.000005e+00,0.536350,9287.168077,935.610297,1.000005e+00,209.822693,21.566375,22.373529,9.846012,645.565179,1.720124,27758.278975,2.580036,0.381138,0.949577,2.699128,219.150021
min,NaN,1.000000,-4.004738e-01,1.000000,0.000000,0.000000,-2.014857e+01,0.850000,0.000000,1.000000,5.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,NaN,3.000000,-3.921993e-01,1.000000,7995.000000,526.000000,-2.490903e-01,45.990000,13.900000,10.000000,43.000000,357.000000,1.000000,23986.000000,3.000000,1.000000,1.000000,1.000000,60.230000
50%,NaN,3.000000,-3.872453e-01,1.000000,16249.000000,1342.000000,1.654726e-01,88.000000,17.270000,28.000000,51.000000,621.000000,2.000000,48053.000000,5.000000,1.000000,1.000000,2.000000,103.370000
75%,NaN,3.000000,1.583079e-01,1.000000,23920.000000,2260.000000,5.498883e-01,149.900000,23.920000,38.000000,57.000000,982.000000,3.000000,72088.000000,7.000000,1.000000,2.000000,4.000000,174.880000


In [ ]:
# assert no null values
assert bookings_payment_df.isnull().sum().sum() == 0

In [ ]:
# train_booking_df contains bookings_df with booking_id in train_data
train_booking_df = bookings_payment_df[bookings_payment_df['booking_id'].isin(train_data['booking_id'])]

# create X_train and Y_train
train_booking_df = train_booking_df.sort_values(by=['booking_id'])
X_train = train_booking_df.drop(['booking_id'], axis=1)
train_data = train_data.sort_values(by=['booking_id'])

# take only unique values
train_data = train_data.drop_duplicates(subset=['booking_id'])
Y_train = train_data['rating_score']

print(X_train.shape)
print(Y_train.shape)

(49868, 18)
(49868,)


In [ ]:
# # data processing 
# 'learning_rate': [0.05],
#     'max_iter': [500],
#     'max_leaf_nodes': [31],
#     'max_depth': [7],
#     'l2_regularization': [0.2],
#     'early_stopping': [False],
#     'validation_fraction': [0.2],
#     'loss': ['squared_error']
# histgradboosting

params = {'learning_rate': 0.05, 'max_iter': 500, 'max_leaf_nodes': 31, 'max_depth': 7, 'l2_regularization': 0.2, 'loss': 'squared_error'}

model = HistGradientBoostingRegressor(**params)

model.fit(X_train, Y_train)


HistGradientBoostingRegressor(l2_regularization=0.2, learning_rate=0.05,
                              max_depth=7, max_iter=500)

In [ ]:
X_train_2, X_valid, Y_train_2, Y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# create ElasticNetCV model with best parameters
elastic_net_cv = HistGradientBoostingRegressor(**params)

# fit the model
elastic_net_cv.fit(X_train_2, Y_train_2)

train_mse = mean_squared_error(Y_train_2, elastic_net_cv.predict(X_train_2))
val_mse = mean_squared_error(Y_valid, elastic_net_cv.predict(X_valid))

print('Train MSE: ', train_mse)
print('Validation MSE: ', val_mse)

Train MSE:  1.2648209774959138
Validation MSE:  1.332180150167379


In [ ]:
# use best parameters to train model
# use best params
model = HistGradientBoostingRegressor(**params)
# fit model
model.fit(X_train, Y_train)

train_mse = mean_squared_error(Y_train, model.predict(X_train))
print("train_mse: {}".format(train_mse))

test_booking_df = bookings_payment_df[bookings_payment_df['booking_id'].isin(test_data['booking_id'])]

# create X_test
test_booking_df = test_booking_df.sort_values(by=['booking_id'])
X_test = test_booking_df.drop(['booking_id'], axis=1)

Y_test_pred = model.predict(X_test)

# prepare submission file
submission = pd.DataFrame()
submission['booking_id'] = test_booking_df['booking_id']
submission['rating_score'] = Y_test_pred

# change ratings below 0 to 0 and above 5 to 5
submission['rating_score'] = submission['rating_score'].apply(lambda x: 1 if x < 1 else x)
submission['rating_score'] = submission['rating_score'].apply(lambda x: 5 if x > 5 else x)

submission.to_csv('HistGrad-best-something-4.csv', index=False)
submission.describe()

train_mse: 1.297656516495869


,rating_score
count,49079.000000
mean,4.087457
std,0.654156
min,1.317976
25%,4.098420
50%,4.342367
75%,4.424671
max,4.633710
